#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 8: Demanda de bicicletas compartidas. Predicciones con *Recurrent Neural Networks*  </center>

#Introducción

Las Redes Neuronales Recurrentes o *Recurrent Neural Networks* (RNN), fueron diseñadas especialmente para aprender a partir de datos secuenciales como: audio, texto, series temporales, entre muchos otros ejemplos. Para trabajar sobre algunos conceptos fundamentales de estas redes, se recurre nuevamente al conjunto de datos [*Bike Sharing Demand*](https://www.kaggle.com/c/bike-sharing-demand). Como ya se sabe estos datos cuentan con una marca de tiempo, lo cual les da un contexto temporal. Esta información es lo que va a permitir poder tratar a estos datos como secuenciales. 
A nivel del problema la idea es predecir la demanda de bicicletas en el futuro a partir de una secuencia fija de datos del pasado. 


## Objetivos


*   Manipular secuencias de datos.
*   Comparar diferentes enfoques de modelos de RNN para un problema concreto.   

## Formas de trabajo

### Opción 1: Trabajar localmente

Descargar los datos en su máquina personal y trabajar en su propio ambiente de desarrollo. Asumiendo que ya creo un entorno para los talleres anteriores sólo debería installar la librería faltantes.  
 
*conda activate TAA-py38*    
*pip install xgboost seaborn*          
*jupyter-notebook*    

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
*!pip install paquete_faltante*

### Opción 2:  Trabajar en *Colab*. 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2022/blob/main/talleres/taller8_demanda_de_bicicletas_rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a *Archivo-->Guardar una copia en drive*.

En caso de estar trabajando desde un notebook en Colab, deberá:

a) Installar el paquete *kaggle* para acceder a los datos 

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


b) realizar la configuración necesaria para obtener datos desde la plataforma Kaggle. Para ello deberá ir a la página de la competencia y en la sección *data* aceptar los términos. Luego ejecutar la siguiente celda y pasarle el *token* de su usuario (ver comentario en celda).

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# El siguiente archivo solicitado es para habilitar la API de Kaggle en el entorno que está trabajando.
# Este archivo se descarga entrando a su perfíl de Kaggle, en la sección API, presionando donde dice: Create New API Token

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


Una vez guardado el *token* se pueden descargar los datos.

In [ ]:
# Descarga de datos
!kaggle competitions download -c bike-sharing-demand
!unzip bike-sharing-demand.zip
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_theme(style="whitegrid")

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 18.4MB/s]
Archive:  bike-sharing-demand.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_submission = pd.read_csv('sampleSubmission.csv')

# Preprocesamiento de datos

## Parte 1: Datos faltantes
Este conjunto cuenta con datos faltantes lo cual es un problema que no fue tenido en cuenta en los talleres anteriores, pero sí puede ser relevante para trabajar con secuencias. La falta de datos se debe a la inexistencia de filas, tanto en el conjunto de *train* como en el conjunto de *test* (sin tener en cuenta cómo fueron divididos estos conjuntos). 

*   Completar la siguiente función para rellenar los datos faltantes teniendo en cuenta que se tienen datos numéricos y categóricos. (*Sugerencias:* [*pd.DataFrame.interpolate()*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html) y [*pd.DataFrame.fillna()*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html))




In [ ]:
from datetime import datetime
import calendar

def FilledIn(df):
    df_aux = df.copy()
    df_out = pd.DataFrame(columns=df_aux.columns)
    df_aux['datetime'] = pd.to_datetime(df_aux['datetime'])
    df_aux = df_aux.set_index('datetime')
    
    for year in [2011, 2012]:
      for month in range(12):
        start_date = datetime(year, month+1, 1, 0, 0, 0)
        last_day_of_month = calendar.monthrange(year, month+1)[1]
        end_date = datetime(year, month+1, last_day_of_month, 23, 0, 0)
        # Se agregan las marcas de tiempo que faltan
        df_month = df_aux[start_date:end_date]
        df_month = df_month.resample('H').asfreq()
        # Rellenar los datos faltantes===========
          
          #Su código
        
        #========================================
        df_month = df_month.reset_index()
        
        df_out = df_out.append(df_month)
    df_out = df_out.reset_index(drop=True)
    return df_out

## Parte 2: Ingeniería y estandarización de los datos


*   Se aplica la ingeniería de datos utilizada en el Taller 3.
*   Se estandarizan los datos. Se recomienda utilizar [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) para los datos categóricos.



In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

class TimeFeatures(BaseEstimator, TransformerMixin):
  def __init__(self):
    self
  def fit(self, X, y=None):
    # X debe ser un DataFrame
    return self
  def transform(self, X):
    X_aux = X.copy()
    X_aux['datetime'] = pd.to_datetime(X_aux['datetime'])
    X_aux['month'] = X_aux['datetime'].dt.month
    X_aux['weekday'] = X_aux['datetime'].dt.weekday
    X_aux['hour'] = X_aux['datetime'].dt.hour
    X_aux = X_aux.drop('datetime', axis=1)
    return X_aux

cat_features = ['season', 'weather', 'month', 'weekday', 'hour']
num_features = ['temp', 'atemp', 'humidity', 'windspeed']
# [holiday, workingday] ya son onehot
scaler = ColumnTransformer([('cat', OneHotEncoder(), cat_features),
                            ('num', StandardScaler(), num_features),
                            ], remainder='passthrough')

preprocess_pipe = Pipeline([('timefeatures', TimeFeatures()),
                            ('scaler', scaler)])   

df_x = df_full.copy()
print(df_x.shape)
x = preprocess_pipe.fit_transform(df_x.drop('count', axis=1)).toarray()
print(x.shape)

(10944, 10)
(10944, 57)


## Parte 3: Secuencias
Para trabajar con modelos RNN en este tipo de problemas es necesario crear un nuevo *dataset* que incluya las secuencias de entrada al modelo y los valores de *target* para comparar las predicciones. Para esto se sugiere utilizar la función [keras.preprocessing.timeseries_dataset_from_array()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array).

*   Crear dicho conjunto para entrenar un modelo que permita predecir la demanda de la próxima hora, dado que se conocen los datos de las últimas 24 horas. Para evitar introducir ruido en el modelo debido a los saltos entre meses por la extracción de los datos de *test*, se sugiere crear un *tf.data.Dataset* por cada mes e ir concatenandolos a medida que se los va procesando. Puede ser útil reusar el *loop* de la función *FilledIn*. 
*   Se sugiere tomar los primeros 15 días del mes para entrenar y del día 16 al 19 para validar.







# Modelos 
Para las siguientes partes se recomienda utilizar *Comet* para guardar los experimentos. Esta es una buena forma de comparar los diferentes modelos.

## Parte 4: *Naive forecasting*
De manera de tener una referencia de desempeño, en este tipo de problemas se utiliza como medida de base el desempeño de algún predictor muy simple como puede ser *Naive forecasting*. El cual simplemente predice un valor como el valor del dato anterior.

*    Calcular los valores de RMSLE y MAE para el predictor *naive forecasting* simplemente manipulando los indices del vector de *target* preprocesado (dataframe).




## Parte 5: *Seq-to-Vector*
En esta parte se evaluará el desempeño de un modelo simple del tipo secuencia a vector (*seq-to-vector*) sobre el conjunto de la Parte 3. Este tipo de modelos son aquellos que reciben una secuencia a la entrada y devuelve un vector a la salida. Para esta parte se pretende que el mismo cuente con una sola capa recurrente de 64 unidades y una capa densa a la salida.

*   Crear la función de costo de manera que esta calcule el valor de RMSLE. Agregar MAE como métrica.
*   Entrenar el modelo con un optimizador *Adam* y manteniendo el resto de los hiperparámetros por defecto. ¿Cómo es el desempeño con respecto al modelo NF?

*   Justifique la cantidad de parámetros entrenables en base a las matrices de pesos y los vectores de bias de cada capa.
   


##Parte 6: *Seq-to-Seq*
Otro tipo de modelo para atacar el problema anterior son aquellos denominados secuencia a secuencia (*seq-to-seq*). Estos modelos reciben una secuencia a la entrada y devuelven una secuencia a la salida. Para este problema se puede utilizar este modelo para predecir el siguiente valor en cada celda de la red recurrente y no sólo en la última. De esta forma se puede mejorar el desempeño notablemente.
Antes de pasar al entrenamiento se debe modificar el *dataset* de manera que el *target* sea una secuencia de valores.
*    Adaptar el ejemplo 3 de la documentación de la función [keras.preprocessing.timeseries_dataset_from_array()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array) para crear el nuevo *dataset*. Puede ser útil la siguiente función: *tf.data.Dataset.zip((features_dataset, labels_dataset))*.
 
*    Modificar el modelo anterior de manera que la salida de la red sea una secuencia, pero manteniendo el valor de los hiperparámetros.

*    Entrenar el nuevo modelo y comparar con los anteriores. ¿Por qué este modelo logra un desempeño mejor? ¿Por qué se mantiene la cantidad de parámetros entrenables?



## Parte 7: LSTM
En el modelo *seq-to-seq* sustituir la capa *SimpleRNN* por una capa LSTM


*   Entrenar la red y compara el desempeño con los modelos anteriores.
*   Justificar la cantidad de parámetros entrenables (ver la ecuación 15-3 del libro).
*   Cambiar el largo de las secuencias. ¿Cómo varía el desempeño para este modelo? 
*   ¿Es posible reducir la distancia entre las curvas *train* y *validation* con los hiperparámetros *dropout* y *recurrent_dropout*?, ¿A qué parámetros afecta cada uno?. 

*   (Opcional) Probar métodos de regularización como *dropout* y *recurrent_dropout* o penalización de los pesos.





